In [5]:
import torch
import numpy as np
import open3d as o3d

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [7]:
file_path = 'data/SurfaceSamples/ShapeNetV2/02691156/d3b9114df1d8a3388e415c6cf89025f0.ply'

pcd = o3d.io.read_point_cloud(file_path)
pcd = np.asarray(pcd.points)

In [8]:
pcd.shape

(30000, 3)

In [74]:
def get_voxel_indices(points, voxel_resolution, volume_size_half):
    # Normalize points to [0, 1]
    normalized_points = (points[:, :3] + volume_size_half) / (2 * volume_size_half)
    # Convert to voxel indices
    voxel_indices = (normalized_points * voxel_resolution).floor().long()
    return voxel_indices

In [18]:
pcd[:10]

array([[-0.266687 ,  0.014661 ,  0.0714391],
       [-0.097616 ,  0.059629 ,  0.0327963],
       [ 0.129014 ,  0.0490736,  0.0467457],
       [ 0.0137107, -0.0435493,  0.104879 ],
       [-0.003545 ,  0.115527 ,  0.326341 ],
       [-0.25705  ,  0.0389597, -0.0485951],
       [-0.0259822,  0.0429778, -0.025618 ],
       [-0.0517072,  0.050338 ,  0.099694 ],
       [-0.0275918,  0.0404751, -0.163802 ],
       [ 0.257034 ,  0.0455434,  0.0119498]])

In [19]:
normalization_param = np.load('data/NormalizationParameters/ShapeNetV2/02691156/d3b9114df1d8a3388e415c6cf89025f0.npz')

In [25]:
normalization_param.files

['offset', 'scale']

In [26]:
normalization_param['offset']

array([ 8.001924e-06, -6.018100e-02, -1.986000e-02], dtype=float32)

In [27]:
normalization_param['scale']

array([2.367257], dtype=float32)

In [33]:
pcd[:,2].max()

0.407095

In [55]:
sdf = np.load('data/SdfSamples/ShapeNetV2/02691156/1a6ad7a24bb89733f412783097373bdc.npz')
sdf = np.load('data/SdfSamples/ShapeNetV2/04256520/cd249bd432c4bc75b82cf928f6ed5338.npz')

In [58]:
sdf_pos = sdf['pos']
sdf_neg = sdf['neg']

In [59]:
sdf_pos[:,-1].max()

1.1694136

In [60]:
sdf_pos.shape

(196511, 4)

In [61]:
sdf_neg.shape

(288854, 4)

In [71]:
sdf_pos[:,0].min()

-1.1342893

In [85]:
sdf_all = np.vstack((sdf_pos, sdf_neg))
sdf_all.shape

(485365, 4)

In [101]:
sdf_surface = sdf_all[np.abs(sdf_all[:,-1]) < 0.1]
sdf_surface.shape

(446055, 4)

In [102]:
voxel_indices = get_voxel_indices(torch.tensor(sdf_surface[:,:3]), voxel_resolution=32, volume_size_half=1.2)

In [103]:
voxel_indices.shape

torch.Size([446055, 3])

In [131]:
voxel_indices[:10]

tensor([[19, 15, 19],
        [13, 12, 17],
        [ 9, 15, 13],
        [13, 16, 18],
        [22, 12, 11],
        [26, 13, 14],
        [25, 14, 14],
        [14, 13, 10],
        [14, 16, 11],
        [14, 15, 10]])

In [104]:
unique_voxels = torch.unique(voxel_indices, dim=0)

In [105]:
unique_voxels.shape

torch.Size([2582, 3])

In [106]:
unique_voxels

tensor([[ 3, 11, 10],
        [ 3, 11, 11],
        [ 3, 11, 21],
        ...,
        [28, 20, 19],
        [28, 20, 20],
        [28, 20, 21]])

In [6]:
def generate_grid_center_indices(voxel_resolution=32, volume_size_half=1.2):
    '''
    return voxel coordinate in shape [R^3,3], R is cube_size
    '''
    # Divide space into equally spaced subspaces and calculate center position of subspace
    voxel_centers = np.linspace(-volume_size_half, volume_size_half, voxel_resolution, endpoint=False)
    voxel_centers += volume_size_half / voxel_resolution
    # Create grid indices
    all_voxel_centers = np.vstack(np.meshgrid(voxel_centers, voxel_centers, voxel_centers)).reshape(3, -1).T
    all_voxel_centers = torch.tensor(all_voxel_centers).reshape(voxel_resolution, voxel_resolution, voxel_resolution, 3).permute(1,0,2,3)
    return all_voxel_centers

In [211]:
voxel_coordinate = generate_grid_center_indices()

In [217]:
voxel_indices_clamp = voxel_indices.clamp(min=-0.1, max=31.1).long()

In [221]:
surface_voxel_coordinate = voxel_coordinate[voxel_indices[:,0].long(), voxel_indices[:,1].long(), voxel_indices[:,2].long(), :]#.clamp(min=0, max=31)

In [228]:
(surface_voxel_coordinate - sdf_surface[:,:3]).max()

tensor(0.0375, dtype=torch.float64)

In [227]:
(surface_voxel_coordinate[333] - sdf_surface[333][:3]).mean()

tensor(0.0111, dtype=torch.float64)

In [229]:
torch.argmin(surface_voxel_coordinate - sdf_surface[:,:3], dim=0)

tensor([429910, 321594, 340734])

In [231]:
tmp = get_voxel_indices(voxel_coordinate.reshape(-1,3), voxel_resolution=32, volume_size_half=1.2)

In [232]:
voxel_coordinate_back = voxel_coordinate[tmp[:,0].long(), tmp[:,1].long(), tmp[:,2].long(), :]

In [238]:
str(unique_voxels[0].numpy().tolist()).replace('[', '').replace(']','')

'[3, 11, 10]'

In [246]:
from scipy.spatial import cKDTree
sdf_tree = cKDTree(sdf_all[:,:3])
voxel_size = (1.2 * 2) / 32
sdf_grid_radius = 1.5 * voxel_size
results = {}
voxel_coordinates_unique = voxel_coordinate[unique_voxels[:,0].long(), unique_voxels[:,1].long(), unique_voxels[:,2].long(), :]
for cur_voxel_idx, cur_voxel_coordinate in zip(unique_voxels, voxel_coordinates_unique):
    near_sample_indices = sdf_tree.query_ball_point(x=cur_voxel_coordinate.numpy(), r=sdf_grid_radius, p=np.inf)
    key_name =  str(cur_voxel_idx.numpy().tolist()).replace('[', '').replace(']','')
    results[key_name] = near_sample_indices

In [256]:
type(near_sample_indices)

list

In [248]:
results_len = [len(v) for v in results.values()]

In [249]:
max(results_len), min(results_len), sum(results_len) / len(results_len)

(12890, 208, 4654.782726568552)

In [247]:
len(results.keys())

2582

In [245]:
len(results['3, 11, 10'])

321

In [ ]:
num_pts = 0
for k in results.keys():
    num_pts += len(results[k])

In [250]:
key_name

'28, 20, 21'

In [253]:
a = [int(it.replace(' ', '')) for it in key_name.split(',')]

In [254]:
a

[28, 20, 21]

In [1]:
import json
f_name = 'data/SdfSamplesToVoxelIndices/ShapeNetV2/02691156/1021a0914a7207aff927ed529ad90a11_vol-1.2_res-32_expand-1.5.json'
with open(f_name) as f:
    file = json.load(f)

In [3]:
file.keys()

dict_keys(['1, 13, 15', '1, 14, 15', '2, 12, 15', '2, 13, 14', '2, 13, 15', '2, 13, 16', '2, 14, 13', '2, 14, 14', '2, 14, 15', '2, 14, 16', '2, 14, 17', '2, 15, 14', '2, 15, 15', '2, 15, 16', '3, 12, 14', '3, 12, 15', '3, 12, 16', '3, 13, 13', '3, 13, 14', '3, 13, 15', '3, 13, 16', '3, 14, 13', '3, 14, 14', '3, 14, 15', '3, 14, 16', '3, 14, 17', '3, 15, 14', '3, 15, 15', '3, 15, 16', '4, 12, 14', '4, 12, 15', '4, 12, 16', '4, 13, 13', '4, 13, 14', '4, 13, 15', '4, 13, 16', '4, 14, 13', '4, 14, 14', '4, 14, 15', '4, 14, 16', '4, 14, 17', '4, 15, 14', '4, 15, 15', '4, 15, 16', '5, 12, 14', '5, 12, 15', '5, 12, 16', '5, 13, 13', '5, 13, 14', '5, 13, 15', '5, 13, 16', '5, 14, 13', '5, 14, 14', '5, 14, 15', '5, 14, 16', '5, 14, 17', '5, 15, 14', '5, 15, 15', '5, 15, 16', '6, 12, 14', '6, 12, 15', '6, 12, 16', '6, 13, 13', '6, 13, 14', '6, 13, 15', '6, 13, 16', '6, 13, 17', '6, 14, 13', '6, 14, 14', '6, 14, 15', '6, 14, 16', '6, 14, 17', '6, 15, 13', '6, 15, 14', '6, 15, 15', '6, 15, 16', '

In [2]:
len(file['1, 13, 15'])

269

In [13]:
file['1, 13, 15']

[35122,
 203749,
 252981,
 47663,
 206455,
 297889,
 24526,
 21064,
 45003,
 330935,
 45626,
 200851,
 267149,
 14248,
 81836,
 7911,
 99943,
 20157,
 209710,
 111920,
 272601,
 160160,
 227950,
 312138,
 214804,
 112333,
 266421,
 272372,
 70227,
 66031,
 299455,
 188233,
 217252,
 81917,
 250090,
 149250,
 188248,
 81906,
 115372,
 220514,
 186618,
 63126,
 14804,
 284552,
 106623,
 124442,
 172347,
 163171,
 200572,
 191836,
 288363,
 36303,
 236729,
 206135,
 213898,
 206019,
 132650,
 116207,
 51073,
 64675,
 33723,
 85997,
 174334,
 209822,
 128368,
 258844,
 220525,
 202174,
 138654,
 16387,
 103799,
 254624,
 294415,
 203900,
 274432,
 123281,
 199342,
 48258,
 32720,
 314,
 248746,
 58165,
 227295,
 113042,
 160151,
 178995,
 206716,
 143531,
 274966,
 34621,
 224933,
 186283,
 47162,
 79791,
 122617,
 15005,
 224123,
 71495,
 189005,
 273325,
 208016,
 181940,
 299297,
 62581,
 137176,
 90786,
 155777,
 15030,
 253948,
 8464,
 172134,
 256069,
 112702,
 83445,
 21939,
 160200

In [9]:
voxel_coordinate.shape

torch.Size([32, 32, 32, 3])

In [12]:
voxel_coordinate = generate_grid_center_indices()
a = torch.tensor([[0,0,0], [31,31,31], [28,20,21], [10,22,23]]).long()
i, j, k = a[:,0], a[:,1], a[:,2]
voxel_coordinate[i,j,k]

tensor([[-1.1625, -1.1625, -1.1625],
        [ 1.1625,  1.1625,  1.1625],
        [ 0.9375,  0.3375,  0.4125],
        [-0.4125,  0.4875,  0.5625]], dtype=torch.float64)

In [16]:
a = torch.ones(10,4)
b = a[a[:,-1]<0]
len(b)

0

In [2]:
import torch
codes = torch.rand(100,10)
index = torch.tensor([1, 10, 24, 2, 44, 26, 23]).long()
codes[index].shape

torch.Size([7, 10])

In [5]:
import numpy as np
import random
a = np.arange(10)
random.choices(a, k=5)

[8, 8, 1, 4, 6]